In [12]:
#Task1:

from mysql.connector.pooling import MySQLConnectionPool, Error

dbconfig = {
    "database": "little_lemon_db",
    "user": "root",
    "password": "SleepWell888"
}

try:
    pool_b = MySQLConnectionPool(
        pool_name = "pool_b",
        pool_size = 2, #default is 5
        **dbconfig
    )

    # Get the connection from the connection pool "pool_b"
    print(f"{pool_b.pool_name} get connection.")
    connection = pool_b.get_connection()
    
    if connection.is_connected():
        print("Connection is established.")

        # Create a cursor object to interact with the database
        cursor = connection.cursor()
    
    else:
        print("Connection is failed.")

except Error as e:
    print("Error:", e)


pool_b get connection.
Connection is established.


In [13]:
# Task2:

import mysql.connector as connector

try:
    # Establish a database connection
    connection_pool = connector.pooling.MySQLConnectionPool(
        pool_name="pool_b",
        pool_size=2,
        user="root",
        password="SleepWell888",
        database="little_lemon_db"
    )

    # Guest data
    guest_data = [
        {"TableNo": 8, "FirstName": "Anees", "LastName": "Java", "BookingTime": "18:00", "EmployeeID": 6},
        {"TableNo": 5, "FirstName": "Bald", "LastName": "Vin", "BookingTime": "19:00", "EmployeeID": 6},
        {"TableNo": 12, "FirstName": "Jay", "LastName": "Kon", "BookingTime": "19:30", "EmployeeID": 6}
    ]

    if len(guest_data) > connection_pool.pool_size:
        print(f"Error: ({len(guest_data)})guests exceeds pool size({connection_pool.pool_size}).")
    else:
        # Insert guest data into Bookings table
        for guest in guest_data:
            try:
                with connection_pool.get_connection() as connection:
                    #for guest in guest_data:
                    cursor = connection.cursor()
                    insert_query = """
                    INSERT INTO Bookings (TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
                    VALUES (%s, %s, %s, %s, %s)
                    """
                    values = (
                        guest["TableNo"],
                        guest["FirstName"],
                        guest["LastName"],
                        guest["BookingTime"],
                        guest["EmployeeID"]
                    )
                    cursor.execute(insert_query, values)
                    connection.commit()
                    cursor.close()
                    print(f"{guest['FirstName']} {guest['LastName']} booked Table_{guest['TableNo']} at {guest['BookingTime']}.")
            except connector.pooling.PoolError as pe:
                print("Pool Error:", pe)

except connector.Error as e:
    print("Error:", e)

Error: (3)guests exceeds pool size(2).


In [14]:
# Task3:

import mysql.connector as connector

try:
    # Establish a database connection
    connection = connector.connect(
        user="root",
        password="SleepWell888",
        database="little_lemon_db"
    )
    
    if connection.is_connected():
        print("Connection is established.")
        print()

        # cursor get connection
        cursor = connection.cursor()

    # Part 1: Retrieve Little Lemon Manager's information
    select_manager_query = """
    SELECT EmployeeID, Name AS ManagerName
    FROM Employees
    WHERE Role = 'Manager'
    """

    cursor.execute(select_manager_query)
    manager_info = cursor.fetchone()

    if manager_info:
        manager_employee_id = manager_info[0]
        manager_name = manager_info[1]
        print("Little_Lemon Manager:")
        print("EmployeeID:", manager_employee_id)
        print("Name:", manager_name)

    # Part 2: Retrieve Employee with Highest Salary
    select_highest_salary_query = """
    SELECT Name AS EmployeeName, Role
    FROM Employees
    WHERE Annual_Salary = (SELECT MAX(Annual_Salary) FROM Employees)
    """

    cursor.execute(select_highest_salary_query)
    highest_salary_employee = cursor.fetchone()

    if highest_salary_employee:
        highest_salary_name = highest_salary_employee[0]
        highest_salary_role = highest_salary_employee[1]
        print("\nEmployee with Highest Salary:")
        print("Name:", highest_salary_name)
        print("Role:", highest_salary_role)
        print()


    # Part 3: Count Number of Guests Booked between 18:00 and 20:00
    select_guests_between_hours_query = """
    SELECT COUNT(*) AS NumberOfGuests
    FROM Bookings
    WHERE TIME(BookingSlot) BETWEEN '18:00:00' AND '20:00:00'
    """

    cursor.execute(select_guests_between_hours_query)
    number_of_guests = cursor.fetchone()[0]
    print(number_of_guests, "Guests Booked at 18:00~20:00.\n")


    # Part 4: Retrieve Guests Waiting to be Seated Sorted by BookingSlot
    cursor.execute("""
    SELECT
        Bookings.BookingID AS ID,
        CONCAT(GuestFirstName, ' ', GuestLastName) AS GuestName,
        Role AS Employee
    
    FROM Bookings
    LEFT JOIN Employees ON Employees.EmployeeID = Bookings.EmployeeID
    WHERE Employees.Role = "Receptionist"
    ORDER BY BookingSlot DESC;
    """)

    print("The following guests are waiting to be Seated:")
    
    results = cursor.fetchall()
    columns = cursor.column_names
    
    print(columns)
    
    for result in results:
        print(result)

except connector.Error as e:
    print("Error:", e)
    #print("BookingID:", guest[0])
    #print("Name:", guest[1])

#except connector.Error as e:
#    print("Error:", e)

finally:
    # Close the cursor and the connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("\nConnection is closed.")

Connection is established.

Little_Lemon Manager:
EmployeeID: 1
Name: Mario Gollini

Employee with Highest Salary:
Name: Mario Gollini
Role: Manager

4 Guests Booked at 18:00~20:00.

The following guests are waiting to be Seated:
('ID', 'GuestName', 'Employee')

Connection is closed.


In [15]:
#Task4:

import mysql.connector as connector

try:
    # Establish a database connection
    connection = connector.connect(
        user="root",
        password="SleepWell888",
        database="little_lemon_db"
    )

    if connection.is_connected():
        print("Connection is established.")

        # Step_1: Create 'BasicSalesReport' stored procedure
        cursor = connection.cursor()

        # Drop the existing stored procedure if it exists
        drop_procedure_query = "DROP PROCEDURE IF EXISTS BasicSalesReport"
        cursor.execute(drop_procedure_query)

        # Create PeakHours stored procedure

    stored_procedure_query = """
    CREATE PROCEDURE BasicSalesReport()
    BEGIN
        SELECT
            SUM(BillAmount) AS Total_Sale,
            AVG(BillAmount) AS Average_Sale,
            MIN(BillAmount) AS Min_Bill_Paid,
            MAX(BillAmount) AS Max_Bill_Paid
        FROM Orders;
    END
    """
        
    # Run stored procedure
    cursor.execute(stored_procedure_query)

    #******************************************

    # Invoke callproc to call the stored procedure
    cursor.callproc("BasicSalesReport")

    # Fetch the results in a variable called dataset
    results = next(cursor.stored_results())
    dataset = results.fetchall()

    # Extract column names
    for column_id in cursor.stored_results():
        columns = [column[0] for column in column_id.description]

    # Print column names
    print("Today's sales report:")
    for result in results:
        print("\t",columns[0],":",result[0])
        print("\t",columns[1],":",result[1])
        print("\t",columns[2],":",result[2])
        print("\t",columns[3],":",result[3])
    

    # Print sorted data using a loop
    for data in dataset:
        print(data)

except connector.Error as e:
    print("Error:", e)




Connection is established.
Today's sales report:
(Decimal('243'), Decimal('48.6000'), 37, 86)


In [16]:
# Task5:

import mysql.connector.pooling

try:
    # Establish a database connection pool
    pool = mysql.connector.pooling.MySQLConnectionPool(
        pool_name="pool_a",
        pool_size=2,
        user="root",
        password="SleepWell888",
        database="little_lemon_db"
    )

    # Get a connection from the pool
    connection = pool.get_connection()

    if connection.is_connected():
        print("The Connection pool is created with a name:",pool.pool_name)
        print("The pool size is:",pool.pool_size)
        print()

        # Create a buffered cursor
        cursor = connection.cursor(buffered=True)

        # Combine data from Bookings and Employee tables and sort by BookingSlot
        query = """
        SELECT BookingSlot, CONCAT(GuestFirstName, ' ', GuestLastName) AS GuestName,
               CONCAT(Name, ' [', e.Role, ']') AS AssignedTo
        FROM Bookings AS b
        LEFT JOIN Employees AS e ON b.EmployeeID = e.EmployeeID
        ORDER BY b.BookingSlot ASC
        LIMIT 3
        """

        cursor.execute(query)

        # Fetch the results
        upcoming_bookings = cursor.fetchall()

        # Print the results
        #print("\nBookingSlot\n[Guest_name]\n[Assigned to: Employee Name [Employee Role]]\n")
        for booking in upcoming_bookings:
            print(f"BookingSlot {booking[0]}\n        Guest_name: {booking[1]}\n        Assigned to: {booking[2]}\n")

except mysql.connector.Error as e:
    print("Error:", e)
finally:
    # Return the connection back to the pool
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Connection is closed and returned to the pool.")


The Connection pool is created with a name: pool_a
The pool size is: 2

BookingSlot 15:00:00
        Guest_name: Vanessa McCarthy
        Assigned to: Giorgos Dioudis [Head Chef]

BookingSlot 17:30:00
        Guest_name: Marcos Romero
        Assigned to: Fatma Kaya [Assistant Chef]

BookingSlot 18:30:00
        Guest_name: Hiroki Yamane
        Assigned to: Adrian Gollini [Assistant Manager]

Connection is closed and returned to the pool.
